# Installation
## Cytomine:
Using pip, you can do it with these two lines:
~~~
$ curl -s https://packagecloud.io/install/repositories/cytomine-uliege/Cytomine-python-client/script.python.sh | bash
$ pip install cytomine-python-client
~~~


# Import


In [ ]:
from cytomine import Cytomine
from cytomine.models import ProjectCollection, ImageInstanceCollection, AnnotationCollection
import getpass
import shapely
from shapely import wkt
from shapely.affinity import affine_transform
import cv2                             # OpenCV
from imgTools import display, multiDisplay
import os, sys
import logging
import segment_detector as sd
import numpy as np
from metrics_lines import metricNaive, metricFalseNeg, metricFalsePos, metricTot

# Evaluation of line detection

TODO: for Jo : example of the metrics

In [ ]:
import evaluateSegDetec

resp = input("Do you want to evaluate the line detector of part1? This will takes 2-3 minutes. [y, n] : ")
if resp == 'y':
    evaluateSegDetec.evaluate()

# Preprocessing

The processing for the eyes and the soccer image dataset applied is different because of the different objective aims for each of them.

**Eyes**

The aim for this dataset is to isolate the iris of the eye. This is done through multiple steps. The particularity of the iris is that it contains the darkest pixel of the images. So the main procedure will rely on increasing the contrast of the image. 

First, we proceed to a normalisation of the gray-scale image. Then we increase the contrast by multiplying all the grayscale values by a given factor. The factor has been chosen according to the value histogram of the whole dataset with some manual tuning. We had chosen to use a 15 factor because it was a good trade-off for isolating the iris and reduce the negative impact of some group of images with eye-liners which are of the same level of grey or darker than the iris.

Then, we apply a binary threshold by applying the Otsu algorithm to define the threshold. It succeeds globally in splitting the value histogram between the skin/rest of the eye and the iris (potentially eyelashes). These two successive operations are similar to an increase in contrast.  Finally, we finish the preprocessing with dilatation to close the shape of the iris. A close operation wasn't needed because they weren't any isolated small artifact after the increase in contrast.



**Soccer**

The aim for this dataset is to isolate the line of the soccer field the best as we can. This is rather more complex than the preceding context. The end of this process was inspired from a paper from the Sharif University of Technology in Iran. "Automatic Soccer Field Line Recognition by Minimum Information" : http://confnews.um.ac.ir/images/41/conferences/aisp2015/183_3.pdf. 

The following step where apply to the image after blurring to reduce the complexity of some information and uniform the images in its different colour space.

We first take back our HSV mask approch from the first part of this project. By applying an HSV mask, we are able to catch all the pixel corresponding to the green spectrum by determining the range of the Hue value. The range was defined by the histogram and manual adjustment. This first step isolates most of the soccer field.

Then, we keep only the biggest non black 4-way connected component of the images in order to remove all artefact outside the soccer field. Before doing this, we apply a dilatation operation with a horizontal rectangle kernel. Because the central line of the field is composed of white value corresponding to the whole Hue range and some dirt (brown Hue range) could be observed on its border in a considerable number of pixels. This issue splits the soccer field into two 4-way connected components. But the horizontal rectangle dilatation succeeds in connected these two components without connecting them to outsider artefacts. 

When this is done, we add the remaining White value missing from the first mask by applying a range on the saturation and the value, low saturation range [0, 30] and high value [190, 255]. These values were defined by observation on some case because the histogram wasn't helping significantly to identify a pic. After applying the 2nd HSV mask, we proceed again in an isolation of the biggest 4-way connected component.  

The last part is focused on retrieving the lines. The quote paper explains the approach we apply in a small section. The main idea is to proceed to a median filtering in the LAB colour space on the L channel only. The L channel represents luminescence value and is not equivalent to the Grey Scale nor the Value channel of HSV because it applies a nonlinear transformation of the RGB space. This colour space is slightly better for filtering operation because it better dissociates brightness and chromatic data. The median filtering will help to uniform the luminescence over the soccer field and its dimension were chosen to be able to drop the luminescence data from the white line. Then we subtract the image before the operation and the filtered image. After applying a binary threshold (\[0, 10\]\[11, 255\]) and remove small connected component (closing operation could have wipe out thin white lines), we have our final prepossessing image which consists of all bright pixel on the soccer field.

**Data Augmentation**

The objective was to extend the dataset of soccer images because it is by far the most complicated case for the deep learning model. In order to get synthetic images with synthetic annotations, we based our approach on the already preprocessed dataset from the original soccer images dataset. Because the preprocessing parameters are really sensible to a certain level of zoom and we aren't sure of how the preprocess of modified images will result. It is also a good way to save resources and to get as fast as possible new data. The lines are already extracted so we applied changes to these images to simulate plausible contexts. We are using the following affine transformations:

- Horizontal Flip
- Zoom [0.8, 0.95] (Zoom in) & [1.2, 2] (Zoom out)
- Shift according to y axis  +/- [0.05 * height, 0.2 * height]

The parameter used are chosen from the previous range according to a uniform law to not bias the simulation and to get something different from the original prepossessed image. Because shifting according to x and zoom in are excluding ellipses from the bounds, we didn't apply or restraint the factor. We use this new augmented dataset according to the variation in accuracy in the model.
